Network for arteficial data

In [1]:
import pandas as pd
import numpy as np
import torch
from scipy.integrate import odeint

In [9]:
data = pd.read_csv('/workspaces/bio-pinn/Arteficial Data/Data_noisy.csv')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
y0 = np.array([1., 0., 0.2, 0.])
print(data)


     time         ConcA     ConcB         ConcC         ConcD
0     0.0  8.383181e+00  0.700421  7.317251e-01  3.763695e+00
1     0.1  7.381639e+00  2.353448 -1.491672e-01  2.799064e+00
2     0.2  9.669862e+00  4.151044 -5.257714e-01  2.079049e+00
3     0.3  1.068910e+01  6.608603 -1.002894e+00  4.513447e-01
4     0.4  1.318064e+01  7.992387 -2.278684e+00 -1.237712e+00
..    ...           ...       ...           ...           ...
96    9.6  3.236100e+19 -8.284169  3.236100e+19 -3.236100e+19
97    9.7  5.335427e+19 -7.540272  5.335427e+19 -5.335427e+19
98    9.8  8.796632e+19 -8.860039  8.796632e+19 -8.796632e+19
99    9.9  1.450319e+20 -7.734428  1.450319e+20 -1.450319e+20
100  10.0  2.391173e+20 -8.072378  2.391173e+20 -2.391173e+20

[101 rows x 5 columns]


In [3]:
class Net(torch.nn.Module):
    
    def __init__(self, k):
        super().__init__()
        self.actf = torch.tanh
        self.f1 = torch.nn.Linear(1, 100)
        self.f2 = torch.nn.Linear(100, 100)
        self.f3 = torch.nn.Linear(100, 100)
        self.f4 = torch.nn.Linear(100, 4)
        
        self.k1 = torch.tensor(k[0], requires_grad=True).float().to(device)
        self.k2 = torch.tensor(k[1], requires_grad=True).float().to(device)
        self.k3 = torch.tensor(k[2], requires_grad=True).float().to(device)
        self.k4 = torch.tensor(k[3], requires_grad=True).float().to(device)

    def forward(self, x):
        x = self.f1(x)
        x = self.f2(x)
        x = self.f3(x)
        x = self.f4(self.actf(x))
        return x.squeeze()

from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self, in_tensor, out_tensor):
        self.inp = in_tensor
        self.out = out_tensor

    def __len__(self):
        return len(self.inp)

    def __getitem__(self, idx):
        return self.inp[idx], self.out[idx]

In [4]:
from torch.autograd import grad

def phys_loss(inp, out):
    dudt = grad(out, inp, grad_outputs=torch.ones_like(out), create_graph=True, allow_unused=True)[0][:,1]
    dudx = grad(out, inp, grad_outputs=torch.ones_like(out), create_graph=True, allow_unused=True)[0][:,0]
    d2udx2 = grad(dudx, inp, grad_outputs=torch.ones_like(dudx), create_graph=True, allow_unused=True)[0][:,0]
    return torch.nn.MSELoss()(d2udx2, 0.5*dudt)

In [39]:
k = [1., 1., 1., 1.]
from torch.optim import Adam
model_pinn = Net(k).to(device)

epochs = 1000
optimizer_pinn = Adam(model_pinn.parameters(), lr=0.1)
loss_fcn = torch.nn.MSELoss()
df=data[['ConcA','ConcB','ConcC','ConcD']]
tensor = torch.tensor(df.values, dtype=torch.float32, requires_grad=True)
tensor=tensor.reshape(-1,1)
time=torch.tensor(data['time'].values, dtype=torch.float32)


from torch.utils.data import DataLoader
train_in=time
train_out=tensor
train_in.to(device)
train_out.to(device)

train_dataset = MyDataset(train_in, train_out)
train_dataloader = DataLoader(train_dataset, batch_size=102)



for epoch in range(0, epochs):
    for batch_in, batch_out in train_dataloader:
        #batch_in = Variable(batch_in, requires_grad=True)
        batch_in.to(device)
        batch_out.to(device)
        model_pinn.train()
        def closure():
            optimizer_pinn.zero_grad()
            loss = loss_fcn(model_pinn(batch_in), batch_out)
            loss += phys_loss(batch_in, model_pinn(batch_in))
            loss += loss_fcn(model_pinn(train_in_bd), train_out_bd)
            loss.backward()
            return loss
        optimizer_pinn.step(closure)
    model_pinn.eval()
    base_loss = loss_fcn(model_pinn(train_in), train_out)
    phys_loss = phys_loss(train_in, model_pinn(train_in))
    bdry_loss = loss_fcn(model_pinn(train_in_bd), train_out_bd)
    epoch_loss = base + phys + bdry

    print(f'Epoch: {epoch+1} | Loss: {round(float(epoch_loss), 4)} = {round(float(base_loss), 4)} + {round(float(phys_loss), 4)} + {round(float(bdry_loss), 4)}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x101 and 1x100)